#Student Name: Martin Power
#Student ID: 9939245

## Etivity3 : Vectors, Matrics and Tensors


Provide Functions to:
* Return the size of a matrix as a 2-dimensional tuple (matrix_size function)
* Sum and subtract 2 matrices (matrix_add_sub function)
* Multiply two matrices of suitable size (matrix_mul function)


In [ ]:
import unittest

##################################################
# Matrices
##################################################
# 2x2 Matrices
a = ((-2, 7), (9, 4))        # A: 2x2 Matrix
b = ((4, 8), (-6, 1))        # B: 2x2 Matrix
c = ((8, 2), )               # C: 1x2 Row Vector
d = ((-3,), (4,))            # D: 2x1 Column Vector

# 4x4 Matrices (Values from Cormac Lavery's Review)
e = ((4, 1, 3, 2), (3, 7, 4, 1), (9, 2, 8, 3), (1, 1, 5, 6))
f = ((2, 6, 8, 9), (2, 6, 2, 4), (4, 3, 1, 6), (1, 1, 9, 2))
g = ((1,), (2,), (3,), (4,))

##################################################
# Functions
##################################################


def matrix_size(n):
    """Function that accepts a matrix in the form of a list of lists and returns
       a tuple indicating matrix size
       Returns (Number of Rows, Number of Columns)
    """
    nr = len(n)
    nc = len(n[0])
    return (nr, nc)


def matrix_init(size):
    """Function to initialize a matrix. Size received as a tuple of the form
       (Number of Rows, Number of Columns)
       Returns matrix in specified size as a list of lists
    """
    # Code taken from https://www.programiz.com/python-programming/matrix
    nr, nc = size
    n = [0]*nr
    for i in range(nr):
        n[i] = [0]*nc
    return n


def matrix_add_sub(n, m, subtract=False):
    """Function to add or substract two matrices. Operation is specified using
    subtract variable and defaults to addition
    """
    (nr, nc) = matrix_size(n)

    # Matrix Size Mismatch. Addition/Subtraction Not Possible
    if(matrix_size(m)) != (nr, nc):
        raise ValueError("matrix_size_mismatch")
    else:
        result = matrix_init((nr, nc))

    for i in range(nr):
        for j in range(nc):
            if(subtract):
                result[i][j] = n[i][j]-m[i][j]
            else:
                result[i][j] = n[i][j]+m[i][j]

    return tuple(tuple(i) for i in result)


def matrix_mul(n, m):
    """Function to multiply two matrices. If matrix dimensions prevent
    multiplication, ERROR is returned
    """
    (nr_n, nc_n) = matrix_size(n)
    (nr_m, nc_m) = matrix_size(m)

    # Number of Rows in Matrix N must equal Number of Columns in Matrix M
    if(nc_n != nr_m):
        raise ValueError("matrix_dimension_mismatch")
    else:
        result = matrix_init((nr_n, nc_m))

    for i in range(nr_n):
        for j in range(nc_m):
            for k in range(nc_n):  # Could also have used nr_m as nc_n==nr_m
                result[i][j] += n[i][k] * m[k][j]

    return tuple(tuple(i) for i in result)


##################################################
# Unit Test
##################################################


class TestEtivity3(unittest.TestCase):
    # Test 2x2 Matrix Addition and Subtraction
    def test_001(self):
        expected_result = ((2, 15), (3, 5))  # A+B
        self.assertTupleEqual(matrix_add_sub(a, b), expected_result, "A+B Mismatch")

        expected_result = ((-6, -1), (15, 3))  # A-B
        self.assertTupleEqual(matrix_add_sub(a, b, subtract=True), expected_result, "A-B Mismatch")

        # Check that A+C rasies a ValueError
        self.assertRaises(ValueError, lambda: matrix_add_sub(a, c))

    # Test 2x2 Matrix Multiplication
    def test_002(self):
        expected_result = ((-50, -9), (12, 76))  # A*B
        self.assertTupleEqual(matrix_mul(a, b), expected_result, "A*B Mismatch")

        expected_result = ((64, 60), (21, -38))  # B*A
        self.assertTupleEqual(matrix_mul(b, a), expected_result, "B*A Mismatch")

        expected_result = ((2, 64),)  # C*A
        self.assertTupleEqual(matrix_mul(c, a), expected_result, "C*A Mismatch")

        expected_result = ((34,), (-11,))  # A*D
        self.assertTupleEqual(matrix_mul(a, d), expected_result, "A*D Mismatch")

        expected_result = ((-16,),)  # C*D
        self.assertTupleEqual(matrix_mul(c, d), expected_result, "C*D Mismatch")

        expected_result = ((-24, -6), (32, 8))  # D*C
        self.assertTupleEqual(matrix_mul(d, c), expected_result, "D*C Mismatch")

        # Check that A*C raises a value error
        self.assertRaises(ValueError, lambda: matrix_mul(a, c))

    # Test 4x4 Matrix Operations
    def test_003(self):
        # E+F
        expected_result = ((6, 7, 11, 11), (5, 13, 6, 5), (13, 5, 9, 9), (2, 2, 14, 8))
        self.assertTupleEqual(matrix_add_sub(e, f), expected_result, "E+F Mismatch")

        # E-F
        expected_result = ((2, -5, -5, -7), (1, 1, 2, -3), (5, -1, 7, -3), (0, 0, -4, 4))
        self.assertTupleEqual(matrix_add_sub(e, f, subtract=True), expected_result, "E-F Mismatch")

        # E*F
        expected_result = ((24, 41, 55, 62), (37, 73, 51, 81), (57, 93, 111, 143), (30, 33, 69, 55))
        self.assertTupleEqual(matrix_mul(e, f), expected_result, "E*F Mismatch")

        # E*G (Matric by Vector)
        expected_result = ((23,), (33,), (49,), (42,))
        self.assertTupleEqual(matrix_mul(e, g), expected_result, "E*G Mismatch")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
